# Homework 3 - Improving the Pipeline

CAPP 30235 Machine Learning for Public Policy

Jonathan Tan

May 2, 2019

In [234]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [235]:
# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Import pipeline library, hardcoded config file values
import pipeline_library as library
import pipeline_config as config

## Part 1: Coding Assignment 

### 1. Fix and improve the pipeline code you submitted for the last assignment based on the feedback from the TA. if something critical was pointed out in the feedback, you need to fix it. You'll get the last homework back by ends of thursday so you'll still have time before this one is due to address those comments.

I received no feedback from my last assignment, so I have skipped this step.

### 2. Add more classifiers to the pipeline. It should at least have Logistic Regression, K­Nearest Neighbor, Decision Trees, SVM, Random Forests, Boosting, and Bagging. The code should have a parameter for running one or more of these classifiers and your analysis should run all of them.

### 3. Experiment with different parameters for these classifiers (different values of k for example, as well as parameters that other classifiers have). You should look at the sklearn documentation to see what parameter each classifier can take and what the default values sklearn selects. The labs should be helpful here.

### 4. Add additional evaluation metrics that we've covered in class to the pipeline (accuracy, precision at different levels, recall at different levels, F1, area under curve, and precision-recall curves).

### 5. Create temporal validation function in your pipeline that can create training and test sets over time. You can choose the length of these splits based on analyzing the data. For example, the test sets could be six months long and the training sets could be all the data before each test set.

## Part 2: Analysis


In [236]:
# Import data for exercise
df = library.read_data(config.DATA_PATH)
print(df.shape)
df.head()

(124976, 26)


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Visual Arts,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,4/14/13,5/2/13
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literature & Writing,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,4/7/12,4/18/12
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,Social Sciences,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,1/30/12,4/15/12
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,10/11/12,12/5/12
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literature & Writing,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,1/8/13,3/25/13


Data cleaning steps:
1. Define label: funded_in_60_days
2. Clean missing data
3. Make binary data into true binary
4. Make categorical data into dummies
5. Drop features that are too collinear

In [237]:
# 1. Define label: funded_in_60_days

df['date_posted'] = pd.to_datetime(df['date_posted'], format='%m/%d/%y')
df['datefullyfunded'] = pd.to_datetime(df['datefullyfunded'], format='%m/%d/%y')
df['fully_funded_in_60_days'] = \
    (df['datefullyfunded'] - df['date_posted'] <= pd.to_timedelta(60, unit='days')).astype('int')
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded,fully_funded_in_60_days
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,2013-05-02,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,2012-04-18,1
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,2012-04-15,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,2012-12-05,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,2013-03-25,0


In [238]:
# 2. Clean missing data
df.isnull().sum(axis=0)

# school_ncesid - 9233 missing, is an ID column, can drop
# school_metro - 15224 missing, categorical - replace with category for other schools sharing the same district?
# school_district - 172 missing, categorical - leave as is
# primary_focus_subject - 15 missing, categorical - leave as is
# primary_focus_area - 15 missing, categorical - leave as is
# secondary_focus_subject - 40556 missing, categorical - leave as is
# secondary_focus_area - 40556 missing, categorical - leave as is
# resource_type - 17 missing, categorical - leave as is
# grade_level - 3 missing, leave as is
# students_reached - 59 missing, integer, fill with mean

df['students_reached'] = library.fill_missing(df['students_reached'])

In [239]:
# 3. Make binary data into true binary
binary_features = ['school_charter', 'school_magnet', 'eligible_double_your_impact_match']

for i in binary_features:
    df[i] = np.where(df[i] == 't', 1, 0)

df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded,fully_funded_in_60_days
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,0,2013-04-14,2013-05-02,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,1,2012-04-07,2012-04-18,1
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,0,2012-01-30,2012-04-15,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,0,2012-10-11,2012-12-05,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,0,2013-01-08,2013-03-25,0


In [240]:
# 4. Make categorical data into dummies

# CATEGORICAL FEATURES 
categorical_features = ['school_city', 'school_state', 'school_metro', 'school_district', \
                        'school_county', 'teacher_prefix', 'resource_type', \
                        'primary_focus_subject', 'primary_focus_area', \
                        'secondary_focus_subject', 'secondary_focus_area']

# for i in categorical_features:
#     df = library.make_dummy_vars(df, i)
df = df.drop(labels=categorical_features, axis=1)

# ORDINAL FEATURES
ordinal_features = ['poverty_level', 'grade_level']

for i in ordinal_features:
    df = library.make_dummy_vars(df, i)

In [241]:
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_charter,school_magnet,total_price_including_optional_support,students_reached,...,datefullyfunded,fully_funded_in_60_days,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,0,0,1498.61,31.0,...,2013-05-02,1,0,1,0,0,0,0,0,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,0,0,282.47,28.0,...,2012-04-18,1,0,1,0,0,1,0,0,0
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,0,0,1012.38,56.0,...,2012-04-15,0,1,0,0,0,1,0,0,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,0,1,175.33,23.0,...,2012-12-05,1,1,0,0,0,0,0,0,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,0,0,3591.11,150.0,...,2013-03-25,0,1,0,0,0,0,0,0,1


In [242]:
df.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_charter',
       'school_magnet', 'total_price_including_optional_support',
       'students_reached', 'eligible_double_your_impact_match', 'date_posted',
       'datefullyfunded', 'fully_funded_in_60_days',
       'poverty_level_high poverty', 'poverty_level_highest poverty',
       'poverty_level_low poverty', 'poverty_level_moderate poverty',
       'grade_level_Grades 3-5', 'grade_level_Grades 6-8',
       'grade_level_Grades 9-12', 'grade_level_Grades PreK-2'],
      dtype='object')

In [243]:
# 5. Drop features that are too collinear

# library.plot_distributions(df)

In [244]:
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_charter,school_magnet,total_price_including_optional_support,students_reached,...,datefullyfunded,fully_funded_in_60_days,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,0,0,1498.61,31.0,...,2013-05-02,1,0,1,0,0,0,0,0,1
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,0,0,282.47,28.0,...,2012-04-18,1,0,1,0,0,1,0,0,0
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,0,0,1012.38,56.0,...,2012-04-15,0,1,0,0,0,1,0,0,0
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,0,1,175.33,23.0,...,2012-12-05,1,1,0,0,0,0,0,0,1
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,0,0,3591.11,150.0,...,2013-03-25,0,1,0,0,0,0,0,0,1


Model building and validation steps:
1. Split data into test and train sets, dropping extraneous ID columns
2. Train series of classifiers on training data
3. Evaluate accuracy of classifiers on test data

In [245]:
# Split data into test and train sets, dropping extraneous ID columns
df = df.drop(labels=config.ID_COLS, axis=1)
df = df.drop(labels=['date_posted', 'datefullyfunded'], axis=1)
df.head()

,school_latitude,school_longitude,school_charter,school_magnet,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,fully_funded_in_60_days,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
0,41.807654,-87.673257,0,0,1498.61,31.0,0,1,0,1,0,0,0,0,0,1
1,34.296596,-119.296596,0,0,282.47,28.0,1,1,0,1,0,0,1,0,0,0
2,34.078625,-118.257834,0,0,1012.38,56.0,0,0,1,0,0,0,1,0,0,0
3,40.687286,-73.988217,0,1,175.33,23.0,0,1,1,0,0,0,0,0,0,1
4,40.793018,-73.205635,0,0,3591.11,150.0,0,0,1,0,0,0,0,0,0,1


In [ ]:



x_train, x_test, y_train, y_test = library.split_data(df, 
                                                      label=config.LABEL)

In [192]:
# Train series of classifiers on training data
trained = library.train_classifier(x_train, y_train, param_dict=config.MODEL_PARAMS)
trained

Training lr with params {'penalty': 'l1', 'solver': 'liblinear', 'random_state': 0}
Training knn with params {'weights': 'distance'}
Training dt with params {'criterion': 'entropy', 'max_features': 'auto', 'random_state': 0}
Training svm with params {'penalty': 'l1', 'dual': False, 'C': 1.0, 'random_state': 0}
Training rf with params {'criterion': 'entropy', 'max_features': 'auto', 'random_state': 0}


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training boost with params {'random_state': 0}
Training bag with params {'random_state': 0}


{'lr': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l1', random_state=0, solver='liblinear',
           tol=0.0001, verbose=0, warm_start=False),
 'knn': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=None, n_neighbors=5, p=2,
            weights='distance'),
 'dt': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
             max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=0,
             splitter='best'),
 'svm': LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l1', 

In [193]:
# Evaluate accuracy of classifiers on test data
library.validate_classifier(x_test, y_test, trained)

Validating lr
Validating knn
Validating dt
Validating svm
Validating rf
Validating boost
Validating bag


,classifier,accuracy,precision,recall,f1
0,lr,0.711786,0.714953,0.990684,0.830531
1,knn,0.667965,0.736619,0.831562,0.781216
2,dt,0.649348,0.753670,0.754826,0.754248
3,svm,0.712480,0.713803,0.996034,0.831626
4,rf,0.702291,0.743920,0.888095,0.809639
5,boost,0.714267,0.725025,0.965280,0.828078
6,bag,0.701411,0.746657,0.879602,0.807696
